## Incorporazioni

Nel nostro esempio precedente, abbiamo lavorato su vettori bag-of-words ad alta dimensionalità con lunghezza `vocab_size`, e stavamo convertendo esplicitamente da vettori di rappresentazione posizionale a bassa dimensionalità in rappresentazioni sparse one-hot. Questa rappresentazione one-hot non è efficiente in termini di memoria, inoltre, ogni parola viene trattata indipendentemente dalle altre, ovvero i vettori codificati one-hot non esprimono alcuna somiglianza semantica tra le parole.

In questa unità, continueremo a esplorare il dataset **News AG**. Per iniziare, carichiamo i dati e recuperiamo alcune definizioni dal notebook precedente.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)
print("Vocab size = ",vocab_size)

Loading dataset...


d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\train.csv: 29.5MB [00:01, 18.8MB/s]                            
d:\WORK\ai-for-beginners\5-NLP\14-Embeddings\data\test.csv: 1.86MB [00:00, 11.2MB/s]                          


Building vocab...
Vocab size =  95812


## Cos'è l'embedding?

L'idea di **embedding** è rappresentare le parole attraverso vettori densi a dimensione ridotta, che in qualche modo riflettono il significato semantico di una parola. Più avanti discuteremo come costruire embedding di parole significativi, ma per ora pensiamo agli embedding come un modo per ridurre la dimensionalità di un vettore di parole.

Quindi, il livello di embedding prende una parola come input e produce un vettore di output con una dimensione specificata `embedding_size`. In un certo senso, è molto simile al livello `Linear`, ma invece di prendere un vettore codificato one-hot, sarà in grado di accettare un numero di parola come input.

Utilizzando il livello di embedding come primo livello nella nostra rete, possiamo passare dal modello bag-of-words al modello **embedding bag**, dove prima convertiamo ogni parola nel nostro testo nel corrispondente embedding e poi calcoliamo una funzione aggregata su tutti questi embedding, come `sum`, `average` o `max`.

![Immagine che mostra un classificatore embedding per cinque parole in sequenza.](../../../../../translated_images/it/embedding-classifier-example.b77f021a7ee67eee.webp)

La nostra rete neurale classificatrice inizierà con un livello di embedding, seguito da un livello di aggregazione e un classificatore lineare sopra di esso:


In [2]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.mean(x,dim=1)
        return self.fc(x)

### Gestire la dimensione variabile delle sequenze

A causa di questa architettura, i minibatch per la nostra rete dovranno essere creati in un certo modo. Nell'unità precedente, utilizzando il bag-of-words, tutti i tensori BoW in un minibatch avevano una dimensione uguale `vocab_size`, indipendentemente dalla lunghezza effettiva della sequenza di testo. Una volta passati agli embedding di parole, ci troveremo con un numero variabile di parole in ogni campione di testo, e quando combiniamo questi campioni in minibatch dovremo applicare un po' di padding.

Questo può essere fatto utilizzando la stessa tecnica di fornire la funzione `collate_fn` alla sorgente dati:


In [3]:
def padify(b):
    # b is the list of tuples of length batch_size
    #   - first element of a tuple = label, 
    #   - second = feature (text sequence)
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # first, compute max length of a sequence in this minibatch
    l = max(map(len,v))
    return ( # tuple of two tensors - labels and features
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v])
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)

### Addestrare il classificatore di embedding

Ora che abbiamo definito un dataloader adeguato, possiamo addestrare il modello utilizzando la funzione di training che abbiamo definito nell'unità precedente:


In [4]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=1, epoch_size=25000)

3200: acc=0.6415625
6400: acc=0.6865625
9600: acc=0.7103125
12800: acc=0.726953125
16000: acc=0.739375
19200: acc=0.75046875
22400: acc=0.7572321428571429


(0.889799795315499, 0.7623160588611644)

> **Nota**: Qui stiamo addestrando solo per 25k record (meno di un'epoca completa) per risparmiare tempo, ma puoi continuare l'addestramento, scrivere una funzione per addestrare per diverse epoche e sperimentare con il parametro del tasso di apprendimento per ottenere una maggiore accuratezza. Dovresti essere in grado di raggiungere un'accuratezza di circa il 90%.


### Livello EmbeddingBag e Rappresentazione di Sequenze a Lunghezza Variabile

Nell'architettura precedente, era necessario riempire tutte le sequenze fino alla stessa lunghezza per adattarle a un minibatch. Questo non è il modo più efficiente per rappresentare sequenze a lunghezza variabile - un altro approccio sarebbe utilizzare un vettore di **offset**, che contiene gli offset di tutte le sequenze memorizzate in un unico grande vettore.

![Immagine che mostra una rappresentazione di sequenza con offset](../../../../../translated_images/it/offset-sequence-representation.eb73fcefb29b46ee.webp)

> **Nota**: Nell'immagine sopra, mostriamo una sequenza di caratteri, ma nel nostro esempio stiamo lavorando con sequenze di parole. Tuttavia, il principio generale di rappresentare le sequenze con un vettore di offset rimane lo stesso.

Per lavorare con la rappresentazione tramite offset, utilizziamo il livello [`EmbeddingBag`](https://pytorch.org/docs/stable/generated/torch.nn.EmbeddingBag.html). È simile a `Embedding`, ma prende come input un vettore di contenuto e un vettore di offset, e include anche uno strato di aggregazione, che può essere `mean`, `sum` o `max`.

Ecco una rete modificata che utilizza `EmbeddingBag`:


In [5]:
class EmbedClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = torch.nn.EmbeddingBag(vocab_size, embed_dim)
        self.fc = torch.nn.Linear(embed_dim, num_class)

    def forward(self, text, off):
        x = self.embedding(text, off)
        return self.fc(x)

Per preparare il dataset per l'addestramento, dobbiamo fornire una funzione di conversione che preparerà il vettore di offset:


In [6]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1])) for t in b]
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)

Nota che, a differenza di tutti gli esempi precedenti, la nostra rete ora accetta due parametri: il vettore dei dati e il vettore degli offset, che sono di dimensioni diverse. Allo stesso modo, il nostro data loader ci fornisce 3 valori invece di 2: sia il vettore dei testi che quello degli offset sono forniti come caratteristiche. Pertanto, dobbiamo leggermente adattare la nostra funzione di training per gestire questa situazione:


In [7]:
net = EmbedClassifier(vocab_size,32,len(classes)).to(device)

def train_epoch_emb(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.CrossEntropyLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    loss_fn = loss_fn.to(device)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,text,off in dataloader:
        optimizer.zero_grad()
        labels,text,off = labels.to(device), text.to(device), off.to(device)
        out = net(text, off)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count


train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6153125
6400: acc=0.6615625
9600: acc=0.6932291666666667
12800: acc=0.715078125
16000: acc=0.7270625
19200: acc=0.7382291666666667
22400: acc=0.7486160714285715


(22.771553103007037, 0.7551983365323096)

## Semantic Embeddings: Word2Vec

Nel nostro esempio precedente, il livello di embedding del modello ha imparato a mappare le parole in rappresentazioni vettoriali, tuttavia questa rappresentazione non aveva molto significato semantico. Sarebbe utile apprendere una rappresentazione vettoriale tale che parole simili o sinonimi corrispondano a vettori vicini tra loro in termini di una certa distanza vettoriale (ad esempio, distanza euclidea).

Per fare ciò, dobbiamo pre-addestrare il nostro modello di embedding su una vasta collezione di testi in un modo specifico. Uno dei primi metodi per addestrare embedding semantici si chiama [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Si basa su due principali architetture utilizzate per produrre una rappresentazione distribuita delle parole:

 - **Continuous bag-of-words** (CBoW) — in questa architettura, addestriamo il modello a prevedere una parola dal contesto circostante. Dato l'ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, l'obiettivo del modello è prevedere $W_0$ a partire da $(W_{-2},W_{-1},W_1,W_2)$.
 - **Continuous skip-gram** è l'opposto del CBoW. Il modello utilizza la finestra di parole di contesto circostanti per prevedere la parola corrente.

CBoW è più veloce, mentre skip-gram è più lento, ma rappresenta meglio le parole meno frequenti.

![Immagine che mostra entrambi gli algoritmi CBoW e Skip-Gram per convertire parole in vettori.](../../../../../translated_images/it/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Per sperimentare con embedding word2vec pre-addestrati sul dataset Google News, possiamo utilizzare la libreria **gensim**. Di seguito troviamo le parole più simili a 'neural'.

> **Nota:** Quando crei per la prima volta i vettori di parole, il download potrebbe richiedere un po' di tempo!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [9]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Possiamo anche calcolare gli embedding vettoriali dalla parola, da utilizzare nell'addestramento del modello di classificazione (mostriamo solo i primi 20 componenti del vettore per chiarezza):


In [10]:
w2v.word_vec('play')[:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

La cosa fantastica degli embedding semantici è che puoi manipolare la codifica dei vettori per cambiare la semantica. Ad esempio, possiamo chiedere di trovare una parola, la cui rappresentazione vettoriale sia il più vicino possibile alle parole *re* e *donna*, e il più lontano possibile dalla parola *uomo*:


In [10]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Sia CBoW che Skip-Grams sono embedding "predittivi", poiché tengono conto solo dei contesti locali. Word2Vec non sfrutta il contesto globale.

**FastText** si basa su Word2Vec imparando rappresentazioni vettoriali per ogni parola e per i n-grammi di caratteri trovati all'interno di ciascuna parola. I valori delle rappresentazioni vengono poi mediati in un unico vettore a ogni passo di addestramento. Sebbene ciò aggiunga un notevole carico computazionale durante il pre-addestramento, consente agli embedding di parole di codificare informazioni sui sotto-elementi delle parole.

Un altro metodo, **GloVe**, sfrutta l'idea della matrice di co-occorrenza e utilizza metodi neurali per decomporre la matrice di co-occorrenza in vettori di parole più espressivi e non lineari.

Puoi sperimentare con l'esempio cambiando gli embedding in FastText e GloVe, poiché gensim supporta diversi modelli di embedding di parole.


## Utilizzo di Embedding Pre-Addestrati in PyTorch

Possiamo modificare l'esempio sopra per pre-popolare la matrice nel nostro livello di embedding con embedding semantici, come Word2Vec. Dobbiamo tenere conto del fatto che i vocabolari degli embedding pre-addestrati e del nostro corpus di testo probabilmente non corrisponderanno, quindi inizializzeremo i pesi per le parole mancanti con valori casuali:


In [11]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

net = EmbedClassifier(vocab_size,embed_size,len(classes))

print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab.get_itos()):
    try:
        net.embedding.weight[i].data = torch.tensor(w2v.get_vector(w))
        found+=1
    except:
        net.embedding.weight[i].data = torch.normal(0.0,1.0,(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")
net = net.to(device)

Embedding size: 300
Populating matrix, this will take some time...Done, found 41080 words, 54732 words missing


In [12]:
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6359375
6400: acc=0.68109375
9600: acc=0.7067708333333333
12800: acc=0.723671875
16000: acc=0.73625
19200: acc=0.7463541666666667
22400: acc=0.7560714285714286


(214.1013875559821, 0.7626759436980166)

Nel nostro caso, non osserviamo un grande aumento di accuratezza, probabilmente a causa dei vocabolari piuttosto diversi.  
Per superare il problema dei vocabolari differenti, possiamo utilizzare una delle seguenti soluzioni:  
* Riaddestrare il modello word2vec sul nostro vocabolario  
* Caricare il nostro dataset utilizzando il vocabolario del modello word2vec pre-addestrato. Il vocabolario utilizzato per caricare il dataset può essere specificato durante il caricamento.  

L'ultimo approccio sembra più semplice, soprattutto perché il framework `torchtext` di PyTorch contiene un supporto integrato per gli embeddings. Possiamo, ad esempio, istanziare un vocabolario basato su GloVe nel seguente modo:  


In [14]:
vocab = torchtext.vocab.GloVe(name='6B', dim=50)

100%|█████████▉| 399999/400000 [00:15<00:00, 25411.14it/s]


Il vocabolario caricato ha le seguenti operazioni di base:
* Il dizionario `vocab.stoi` ci permette di convertire una parola nel suo indice nel dizionario
* `vocab.itos` fa l'opposto - converte un numero in una parola
* `vocab.vectors` è l'array di vettori di embedding, quindi per ottenere l'embedding di una parola `s` dobbiamo usare `vocab.vectors[vocab.stoi[s]]`

Ecco un esempio di manipolazione degli embedding per dimostrare l'equazione **kind-man+woman = queen** (ho dovuto modificare un po' il coefficiente per farlo funzionare):


In [15]:
# get the vector corresponding to kind-man+woman
qvec = vocab.vectors[vocab.stoi['king']]-vocab.vectors[vocab.stoi['man']]+1.3*vocab.vectors[vocab.stoi['woman']]
# find the index of the closest embedding vector 
d = torch.sum((vocab.vectors-qvec)**2,dim=1)
min_idx = torch.argmin(d)
# find the corresponding word
vocab.itos[min_idx]

'queen'

Per addestrare il classificatore utilizzando quei embeddings, dobbiamo prima codificare il nostro dataset utilizzando il vocabolario GloVe:


In [16]:
def offsetify(b):
    # first, compute data tensor from all sequences
    x = [torch.tensor(encode(t[1],voc=vocab)) for t in b] # pass the instance of vocab to encode function!
    # now, compute the offsets by accumulating the tensor of sequence lengths
    o = [0] + [len(t) for t in x]
    o = torch.tensor(o[:-1]).cumsum(dim=0)
    return ( 
        torch.LongTensor([t[0]-1 for t in b]), # labels
        torch.cat(x), # text 
        o
    )

Come abbiamo visto sopra, tutti gli embedding dei vettori sono memorizzati nella matrice `vocab.vectors`. Questo rende estremamente facile caricare quei pesi nei pesi del livello di embedding utilizzando una semplice copia:


In [17]:
net = EmbedClassifier(len(vocab),len(vocab.vectors[0]),len(classes))
net.embedding.weight.data = vocab.vectors
net = net.to(device)

In [18]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=offsetify, shuffle=True)
train_epoch_emb(net,train_loader, lr=4, epoch_size=25000)

3200: acc=0.6271875
6400: acc=0.68078125
9600: acc=0.7030208333333333
12800: acc=0.71984375
16000: acc=0.7346875
19200: acc=0.7455729166666667
22400: acc=0.7529464285714286


(35.53972978646833, 0.7575175943698017)

Uno dei motivi per cui non stiamo osservando un aumento significativo dell'accuratezza è dovuto al fatto che alcune parole del nostro dataset sono assenti nel vocabolario GloVe pre-addestrato e, di conseguenza, vengono essenzialmente ignorate. Per superare questo problema, possiamo addestrare i nostri embedding sul nostro dataset.


## Contesti Contestuali

Una delle principali limitazioni delle rappresentazioni di embedding pre-addestrati tradizionali, come Word2Vec, è il problema della disambiguazione del significato delle parole. Sebbene gli embedding pre-addestrati possano catturare parte del significato delle parole nel contesto, ogni possibile significato di una parola viene codificato nello stesso embedding. Questo può causare problemi nei modelli a valle, poiché molte parole, come la parola 'play', hanno significati diversi a seconda del contesto in cui vengono utilizzate.

Ad esempio, la parola 'play' in queste due frasi ha significati piuttosto diversi:
- Sono andato a vedere una **play** a teatro.
- John vuole **play** con i suoi amici.

Gli embedding pre-addestrati sopra rappresentano entrambi i significati della parola 'play' nello stesso embedding. Per superare questa limitazione, dobbiamo costruire embedding basati sul **modello linguistico**, che è addestrato su un ampio corpus di testo e *sa* come le parole possono essere combinate in contesti diversi. Discutere degli embedding contestuali esula dall'ambito di questo tutorial, ma torneremo sull'argomento quando parleremo dei modelli linguistici nella prossima unità.



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
